# Distance_detection_method


In [175]:
%matplotlib inline
import numpy as np
import cv2
from velocity import VeloEval
import copy
import math
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn import preprocessing, datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import math
import os


In [158]:
dataset_path = 'benchmark_velocity_train/clips/'
folder_path = os.listdir(dataset_path)

annotations = [os.path.join(dataset_path, x, 'annotation.json') for x in folder_path]
annotations.remove('benchmark_velocity_train/clips/.DS_Store/annotation.json')
annotations.sort() #sort it so that we can read images in order
gt = VeloEval.load_annotation(annotations)


Finished loading 1074 annotations.


In [159]:
images_training_paths = []
for x in folder_path:
    image_path = "benchmark_velocity_train/clips/"+x+"/imgs/040.jpg"
    images_training_paths.append(image_path)
images_training_paths.remove('benchmark_velocity_train/clips/.DS_Store/imgs/040.jpg')
images_training_paths.sort()
len(images_training_paths)

1074

In [242]:
dataset_path_t = 'benchmark_velocity_test/clips/'
folder_path_t = os.listdir(dataset_path_t)

annotations_t = [os.path.join('.','gt.json')]
#annotations_t.remove('benchmark_velocity_test/clips/.DS_Store/annotation.json')
#annotations_t.sort() #sort it so that we can read images in order
gt_t = VeloEval.load_json_file(annotations_t)[0]
len(gt_t)

269

In [239]:
images_testing_paths = []
for x in folder_path_t:
    image_path = "benchmark_velocity_test/clips/"+x+"/imgs/040.jpg"
    images_testing_paths.append(image_path)
images_testing_paths.remove('benchmark_velocity_test/clips/.DS_Store/imgs/040.jpg')
images_testing_paths.sort()
len(images_testing_paths)

269

In [399]:
gt_t[10]

[{'velocity': [-0.9686020636, -0.0679646536],
  'bbox': {'top': 345.3848571777,
   'right': 828.7849731445,
   'left': 756.0120849609,
   'bottom': 388.9237670898},
  'position': [23.4500563918, 2.2509998609]}]

## Expr 1: try features, x1, y1, pixel_w, pixel_h and predict w, and then apply triangle similarity

In [400]:
def get_v(flag, gt):
    if flag == 0:
        #testing data
        y1 = gt['bbox']['top']
        x2 = gt['bbox']['right']
        x1 = gt['bbox']['left']
        y2 = gt['bbox']['bottom']
        z0 = gt['position'][0]
        x0 = gt['position'][1]
    else:
        #training
        y1 = gt['bbox'][0][0]
        x1 = gt['bbox'][0][1]
        y2 = gt['bbox'][0][2]
        x2 = gt['bbox'][0][3]
        z0 = gt['position'][0]
        x0 = gt['position'][1]
    return (x1,x2,y1,y2,z0,x0)
        
        

In [401]:
def get_car_width(gt,flag=1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars.
    Output is weights list of all cars detected in the input image'''
   
    feature_list = []
    fx = 714.1526
    
    for i in range(len(gt)):
        if flag == 0:
            x1,x2,y1,y2 = get_v(0,gt[i])[0:4]
        else:
            x1,x2,y1,y2 = get_v(1,gt[i])[0:4]


        #print(gt)
        z0 = int(gt[i]['position'][0])
        pixel_w = x2 - x1
        pixel_h = y2 - y1

        #pixel_h = y2 - y1
        w = (pixel_w * z0)/fx
        feature_list.append([x1,y1,pixel_w,pixel_h,w])

        
    return feature_list


In [402]:
all_list = [] #this records all the weights in training data

for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = get_car_width(gt_temp)
    all_list.extend(cars_info)
all_list[:5]

[[784.6638793945,
  340.3418273926,
  83.48986816410002,
  52.985687255799974,
  2.455059648940717],
 [650.6055908203,
  347.4986877441,
  35.13989257809999,
  27.825439453200033,
  1.771380700443574],
 [648.2036132812,
  345.9122314453,
  34.08062744150004,
  26.833465576200012,
  1.717983786510056],
 [924.3724975586,
  315.0008850098,
  333.3609008789,
  205.1636047363,
  2.333961263173305],
 [622.0191040039,
  339.5474853516,
  31.20141601559999,
  25.246826171799967,
  2.3155765992125485]]

In [403]:
training_data = np.asarray(all_list)
train_x = training_data[:,0:4]
train_y = training_data[:,4]

In [404]:
all_testing_list = [] #this records all the weights in training data

for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    #print(gt_temp)
    cars_info = get_car_width(gt_temp,0)
    #print(cars_info)
    all_testing_list.extend(cars_info)


In [405]:
all_testing_list[:5]

[[509.4688110352,
  341.8059692383,
  59.577270507799994,
  42.200958251899976,
  2.419288041136026],
 [755.4901733398,
  334.7809143066,
  71.58563232430004,
  50.512664795000035,
  2.4057255771150325],
 [673.6825561523,
  345.3791503906,
  35.51818847660002,
  27.966430664100017,
  1.8401850999831137],
 [614.9835205078,
  343.7766113281,
  32.9028930664,
  24.905639648500028,
  2.1193412739159667],
 [564.3881225586,
  339.0909729004,
  64.3133544922,
  51.0140991211,
  2.1613314966756407]]

In [406]:
testing_data = np.asarray(all_testing_list)
test_x = testing_data[:,0:4]
test_y = testing_data[:,4]

In [407]:
regr = linear_model.LinearRegression()
regr.fit(train_x,train_y)

LinearRegression()

In [408]:
#Predict the targets
predicted_y =regr.predict(test_x)

#Output the coefficient
print('Coefficients: ', regr.coef_[0], regr.intercept_)
#Calculate the Mean Squared Error
mse = np.mean((predicted_y - test_y) ** 2)
print ("Mean squared error: %.2f" % mse)

Coefficients:  -0.000370201424067597 14.252734275136223
Mean squared error: 0.04


In [409]:
regr.coef_[0],regr.coef_[1],regr.coef_[2],regr.coef_[3],regr.intercept_

(-0.000370201424067597,
 -0.03435506853204499,
 0.02162694987625107,
 -0.03347930649287277,
 14.252734275136223)

In [410]:
predicted_y[0:10],test_y[0:10],test_x[0:10]

(array([2.19697653, 2.32867938, 1.96966503, 2.09236313, 2.07728841,
        1.97771741, 2.19225128, 1.89948709, 2.10076575, 2.5751089 ]),
 array([2.41928804, 2.40572558, 1.8401851 , 2.11934127, 2.1613315 ,
        1.82188188, 2.22905938, 1.69564046, 1.98705781, 2.78285763]),
 array([[509.46881104, 341.80596924,  59.57727051,  42.20095825],
        [755.49017334, 334.78091431,  71.58563232,  50.5126648 ],
        [673.68255615, 345.37915039,  35.51818848,  27.96643066],
        [614.98352051, 343.77661133,  32.90289307,  24.90563965],
        [564.38812256, 339.0909729 ,  64.31335449,  51.01409912],
        [671.20300293, 345.68673706,  26.02203369,  21.30337524],
        [575.43719482, 344.18261719,  31.21350098,  20.85140991],
        [687.64801025, 345.7272644 ,  44.84985352,  35.57901001],
        [745.90063477, 341.47943115,  35.4765625 ,  27.22683716],
        [795.46112061, 332.74090576, 110.41027832,  69.8833313 ]]))

In [411]:
784 * (-0.00037590283643064296) + (-0.03446663268860189) * 340 + 0.021661369247535468 * 84 -0.033546266089523366 * 53 +14.279085928507993

2.307325904669966

In [415]:
def predict_distance_5(gt,flag = 1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars. The cdf of training data and corresponding weights bins
    Output is the difference between the ground truth position and the predicted position'''

    good_z0 = 0
    good_x0 = 0
    
    diff_list = []
    diff_x0_list = []
    fx = 714.1526

    for i in range(len(gt)):
        if flag == 0: #testing data
            x1,x2,y1,y2,z0,x0 = get_v(0,gt[i])
        else:#training data
            x1,x2,y1,y2,z0,x0 = get_v(1,gt[i])


        pixel_w = x2 - x1
        pixel_h = y2 - y1
        w = -0.000370201424067597 * x1-0.03435506853204499 * y1 + 0.02162694987625107 * pixel_w -0.03347930649287277 * pixel_h + 14.252734275136223
        '''(-0.000370201424067597,
            -0.03435506853204499,
            0.02162694987625107,
            -0.03347930649287277,
            14.252734275136223)'''

        pred_z0 = fx * w / pixel_w
        pred_x0 = (x1 - 660) * pred_z0/fx
        diff_x0 = x0 - pred_x0
        diff_z0 = z0-pred_z0

        if abs(diff_z0)/z0 < 0.10:
            good_z0 +=1
        if abs(diff_x0)/x0 < 0.10:
            good_x0 +=1



        diff_x0_list.append(abs(diff_x0))
        diff_list.append(abs(diff_z0))


    return (diff_list,diff_x0_list,good_z0, good_x0)
       


In [416]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = predict_distance_5(gt_temp,1)

    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Train_set: AVG Error Z0 {1:.2f}, Distance_within_10%_diff_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(1,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 1/10 Train_set: AVG Error Z0 2.53, Distance_within_10%_diff_rate: 73.79%, AVG Error X0 0.68, X0_within_10%_rate:72.54%


In [417]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    cars_info = predict_distance_5(gt_temp,0)
    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Test_set: AVG Error Z0 {1:.2f}, Distance_within_10%_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(1,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 1/10 Test_set: AVG Error Z0 2.62, Distance_within_10%_rate: 72.00%, AVG Error X0 0.72, X0_within_10%_rate:70.93%


## Expr 2: using y1, pixel_w, piexel_h, pixel_w * pixel_h to predict w and then apply triangle similarity
- Rationale: based on the coefficients obtained above, it seems that x1 has tiny influence on the result. But we can try using pixel_w and pixel_h which indicates the bbox area as a new feature

In [430]:
def get_car_width_2(gt,flag=1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars.
    Output is weights list of all cars detected in the input image'''
   
    feature_list = []
    fx = 714.1526
    
    for i in range(len(gt)):
        if flag == 0:
            x1,x2,y1,y2 = get_v(0,gt[i])[0:4]
        else:
            x1,x2,y1,y2 = get_v(1,gt[i])[0:4]


        #print(gt)
        z0 = int(gt[i]['position'][0])
        pixel_w = x2 - x1
        pixel_h = y2 - y1

        #pixel_h = y2 - y1
        w = (pixel_w * z0)/fx
        feature_list.append([y1,pixel_w,pixel_h,pixel_w*pixel_h,w])

        
    return feature_list






In [431]:
all_list_2 = [] #this records all the weights in training data

for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = get_car_width_2(gt_temp)
    all_list_2.extend(cars_info)


In [432]:
training_data_2 = np.asarray(all_list_2)
train_x_2 = training_data_2[:,0:4]
train_y_2 = training_data_2[:,4]
train_x_2[0:5],train_y_2[0:5]

(array([[3.40341827e+02, 8.34898682e+01, 5.29856873e+01, 4.42376804e+03],
        [3.47498688e+02, 3.51398926e+01, 2.78254395e+01, 9.77782953e+02],
        [3.45912231e+02, 3.40806274e+01, 2.68334656e+01, 9.14501343e+02],
        [3.15000885e+02, 3.33360901e+02, 2.05163605e+02, 6.83935241e+04],
        [3.39547485e+02, 3.12014160e+01, 2.52468262e+01, 7.87736726e+02]]),
 array([2.45505965, 1.7713807 , 1.71798379, 2.33396126, 2.3155766 ]))

In [433]:
all_testing_list_2 = [] #this records all the weights in training data

for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    cars_info = get_car_width_2(gt_temp,0)
    all_testing_list_2.extend(cars_info)

In [434]:
testing_data_2 = np.asarray(all_testing_list_2)
test_x_2 = testing_data_2[:,0:4]
test_y_2 = testing_data_2[:,4]
test_x_2[0:5],test_y_2[0:5]

(array([[ 341.80596924,   59.57727051,   42.20095825, 2514.21790546],
        [ 334.78091431,   71.58563232,   50.5126648 , 3615.98104974],
        [ 345.37915039,   35.51818848,   27.96643066,  993.31695535],
        [ 343.77661133,   32.90289307,   24.90563965,  819.4675981 ],
        [ 339.0909729 ,   64.31335449,   51.01409912, 3280.88784088]]),
 array([2.41928804, 2.40572558, 1.8401851 , 2.11934127, 2.1613315 ]))

In [435]:
test_y_2.shape,test_x_2.shape

((375,), (375, 4))

In [436]:
regr_2 = linear_model.LinearRegression()
regr_2.fit(train_x_2,train_y_2)
#Predict the targets
predicted_y_2 =regr_2.predict(test_x_2)

#Output the coefficient
print('Coefficients: ', regr_2.coef_[0], regr_2.intercept_)
#Calculate the Mean Squared Error
mse_2 = np.mean((predicted_y_2 - test_y_2) ** 2)
print ("Mean squared error: %.2f" % mse_2)


Coefficients:  -0.030522707866872622 12.566680966429356
Mean squared error: 0.04


In [437]:
regr_2.coef_[0],regr_2.coef_[1],regr_2.coef_[2],regr_2.coef_[3],regr_2.intercept_

(-0.030522707866872622,
 0.019645624118997243,
 -0.026543674714586008,
 -1.3394041753397261e-05,
 12.566680966429356)

In [438]:
predicted_y_2[0:5],test_y_2[0:5],test_x_2[0:5]

(array([2.15042583, 2.365381  , 1.96691467, 2.04802258, 2.08213625]),
 array([2.41928804, 2.40572558, 1.8401851 , 2.11934127, 2.1613315 ]),
 array([[ 341.80596924,   59.57727051,   42.20095825, 2514.21790546],
        [ 334.78091431,   71.58563232,   50.5126648 , 3615.98104974],
        [ 345.37915039,   35.51818848,   27.96643066,  993.31695535],
        [ 343.77661133,   32.90289307,   24.90563965,  819.4675981 ],
        [ 339.0909729 ,   64.31335449,   51.01409912, 3280.88784088]]))

In [471]:
def predict_distance_6(gt,p = 670,flag = 1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars. The cdf of training data and corresponding weights bins
    Output is the difference between the ground truth position and the predicted position'''

    good_z0 = 0
    good_x0 = 0
    
    diff_list = []
    diff_x0_list = []
    fx = 714.1526

    for i in range(len(gt)):
        if flag == 0: #testing data
            x1,x2,y1,y2,z0,x0 = get_v(0,gt[i])
        else:#training data
            x1,x2,y1,y2,z0,x0 = get_v(1,gt[i])


        pixel_w = x2 - x1
        pixel_h = y2 - y1
        w = -0.030522707866872622 * y1 + 0.019645624118997243 * pixel_w -0.026543674714586008* pixel_h + -1.3394041753397261e-05 * pixel_w * pixel_h+ 12.566680966429356
        '''(-0.030522707866872622,
             0.019645624118997243,
             -0.026543674714586008,
             -1.3394041753397261e-05,
             12.566680966429356)'''

        pred_z0 = fx * w / pixel_w
        pred_x0 = (x1 - p) * pred_z0/fx
        diff_x0 = x0 - pred_x0
        diff_z0 = z0-pred_z0

        if abs(diff_z0)/z0 < 0.10:
            good_z0 +=1
        if abs(diff_x0)/x0 < 0.10:
            good_x0 +=1



        diff_x0_list.append(abs(diff_x0))
        diff_list.append(abs(diff_z0))


    return (diff_list,diff_x0_list,good_z0, good_x0)
       

In [472]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = predict_distance_6(gt_temp,1)

    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Train_set: AVG Error Z0 {1:.2f}, Distance_within_10%_diff_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(2,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 2/10 Train_set: AVG Error Z0 2.46, Distance_within_10%_diff_rate: 75.94%, AVG Error X0 32.77, X0_within_10%_rate:64.84%


In [474]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    cars_info = predict_distance_6(gt_temp,flag =0)
    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Test_set: AVG Error Z0 {1:.2f}, Distance_within_10%_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(2,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 2/10 Test_set: AVG Error Z0 2.55, Distance_within_10%_rate: 75.20%, AVG Error X0 0.51, X0_within_10%_rate:74.13%


## Expr 3: using y1, pixel_w, piexel_h, pixel_w * pixel_h to directly predict z0
- Rationale: based on above performance, the below experiment is conducted to see if we can save some computations by directly calculating z0

In [443]:
def get_car_distance(gt,flag=1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars.
    Output is weights list of all cars detected in the input image'''
   
    feature_list = []
    fx = 714.1526
    
    for i in range(len(gt)):
        if flag == 0:
            x1,x2,y1,y2 = get_v(0,gt[i])[0:4]
        else:
            x1,x2,y1,y2 = get_v(1,gt[i])[0:4]


        #print(gt)
        z0 = int(gt[i]['position'][0])
        pixel_w = x2 - x1
        pixel_h = y2 - y1

        feature_list.append([y1,pixel_w,pixel_h,pixel_w*pixel_h,z0])

        
    return feature_list

In [450]:
all_list_3 = []

for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = get_car_distance(gt_temp)
    all_list_3.extend(cars_info)


In [451]:
training_data_3 = np.asarray(all_list_3)
train_x_3 = training_data_3[:,0:4]
train_y_3 = training_data_3[:,4]
train_x_3[0:5],train_y_3[0:5]

(array([[3.40341827e+02, 8.34898682e+01, 5.29856873e+01, 4.42376804e+03],
        [3.47498688e+02, 3.51398926e+01, 2.78254395e+01, 9.77782953e+02],
        [3.45912231e+02, 3.40806274e+01, 2.68334656e+01, 9.14501343e+02],
        [3.15000885e+02, 3.33360901e+02, 2.05163605e+02, 6.83935241e+04],
        [3.39547485e+02, 3.12014160e+01, 2.52468262e+01, 7.87736726e+02]]),
 array([21., 36., 36.,  5., 53.]))

In [452]:
all_testing_list_3 = [] 

for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    cars_info = get_car_distance(gt_temp,0)
    all_testing_list_3.extend(cars_info)

In [453]:
testing_data_3 = np.asarray(all_testing_list_3)
test_x_3 = testing_data_3[:,0:4]
test_y_3 = testing_data_3[:,4]
test_x_3[0:5],test_y_3[0:5]

(array([[ 341.80596924,   59.57727051,   42.20095825, 2514.21790546],
        [ 334.78091431,   71.58563232,   50.5126648 , 3615.98104974],
        [ 345.37915039,   35.51818848,   27.96643066,  993.31695535],
        [ 343.77661133,   32.90289307,   24.90563965,  819.4675981 ],
        [ 339.0909729 ,   64.31335449,   51.01409912, 3280.88784088]]),
 array([29., 24., 37., 46., 24.]))

In [456]:
regr_3 = linear_model.LinearRegression()
regr_3.fit(train_x_3,train_y_3)
#Predict the targets
predicted_y_3 =regr_3.predict(test_x_3)

#Output the coefficient
print('Coefficients: ', regr_3.coef_[0], regr_3.intercept_)
#Calculate the Mean Squared Error
mse_3 = np.mean((predicted_y_3 - test_y_3) ** 2)
print ("Mean squared error: %.2f" % mse_3)


Coefficients:  -0.5484539714702852 256.95697122783037
Mean squared error: 36.45


In [458]:
regr_3.coef_[:],regr_3.intercept_

(array([-0.54845397, -0.05404788, -0.97852039,  0.00222415]),
 256.95697122783037)

In [459]:
def predict_distance_7(gt,flag = 1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars. The cdf of training data and corresponding weights bins
    Output is the difference between the ground truth position and the predicted position'''

    good_z0 = 0
    good_x0 = 0
    
    diff_list = []
    diff_x0_list = []
    fx = 714.1526

    for i in range(len(gt)):
        if flag == 0: #testing data
            x1,x2,y1,y2,z0,x0 = get_v(0,gt[i])
        else:#training data
            x1,x2,y1,y2,z0,x0 = get_v(1,gt[i])


        pixel_w = x2 - x1
        pixel_h = y2 - y1
        #y1,pixel_w,pixel_h,pixel_w*pixel_h,z0
        pred_z0 = -0.54845397 * y1 -0.05404788 * pixel_w -0.97852039* pixel_h + 0.00222415 * pixel_w * pixel_h+ 256.95697122783037
        '''[-0.54845397, -0.05404788, -0.97852039,  0.00222415]),
             256.95697122783037'''

        
        pred_x0 = (x1 - 660) * pred_z0/fx
        diff_x0 = x0 - pred_x0
        diff_z0 = z0-pred_z0

        if abs(diff_z0)/z0 < 0.10:
            good_z0 +=1
        if abs(diff_x0)/x0 < 0.10:
            good_x0 +=1



        diff_x0_list.append(abs(diff_x0))
        diff_list.append(abs(diff_z0))


    return (diff_list,diff_x0_list,good_z0, good_x0)
       

In [460]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_training_paths)):
    gt_temp = gt[i]
    cars_info = predict_distance_7(gt_temp,1)

    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Train_set: AVG Error Z0 {1:.2f}, Distance_within_10%_diff_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(2,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 2/10 Train_set: AVG Error Z0 4.35, Distance_within_10%_diff_rate: 50.62%, AVG Error X0 0.98, X0_within_10%_rate:68.31%


In [461]:
diff = []
diff_x0 = []
count_z0 = 0
count_x0 = 0
for i in range(len(images_testing_paths)):
    gt_temp = gt_t[i]
    cars_info = predict_distance_7(gt_temp,0)
    diff.extend(cars_info[0])
    diff_x0.extend(cars_info[1])
    count_z0 += cars_info[2]
    count_x0 += cars_info[3]

print("EXPR {0}/10 Test_set: AVG Error Z0 {1:.2f}, Distance_within_10%_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_10%_rate:{4:.2f}%".format(2,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



EXPR 2/10 Test_set: AVG Error Z0 4.30, Distance_within_10%_rate: 50.93%, AVG Error X0 0.97, X0_within_10%_rate:67.20%


## Expr 4: probability based

In [464]:
def predict_distance_probability(gt,flag=1):
    '''Input is a image and the ground truth of this image that includes the position and bbox of cars.
    There could be more than one cars.
    Output is the difference between the ground truth position and the predicted position'''
    #cars = []
    error_z0 = 0
    error_x0 = 0
    
    z_predict = 0
#     w_list = []
#     h_list = []
#     w_h_ratio_list = []
    diff_list = []
    diff_x0_list = []
    fx = 714.1526
    fy = 710.3725
    para = 1.17
    for i in range(len(gt)):
        if flag == 0: #testing data
            x1,x2,y1,y2,z0,x0 = get_v(0,gt[i])
        else:#training data
            x1,x2,y1,y2,z0,x0 = get_v(1,gt[i])

        
        pixel_w = x2 - x1
        pixel_h = y2 - y1
        w = (pixel_w * z0)/fx
               
        prob_list = [0.06934306569343066,
                     0.21313868613138687,
                     0.38467153284671535,
                     0.5781021897810219,
                     0.7182481751824817,
                     0.8248175182481752,
                     0.916058394160584,
                     0.9591240875912409,
                     0.9832116788321168,
                     1.0]
        weights_list = [1.45,1.55,1.65,1.75,1.85,1.95,2.05,2.15,2.25,2.35]
        u = np.random.random(1)[0] 
        if u <= prob_list[0]:
            w = weights_list[0]
        else:
            for i in range(9):
                if u <= prob_list[i+1] and u >= prob_list[i]:
                    w = weights_list[i+1]

        if -1.125* x1 + 720 - y1 > 0:
                    
            if y1 > 240:
                w = w / 1.2
                     
 
        if 1.125* x1 - 720 - y1 > 0:
            if y1 <= 240:
                w = w/2
                #print(f"right: top: w{w}")
                
            elif y1 <= 480:
                if y2 <= 480:
                    w = w*1.7
                    #print(f"right: middle small: w{w}")

                else:
                    #print(f"right: middle big: w{w}")
                    w = w / 1.2
            else:
        
                w = w / 1.2
                #print(f"right bottom: w{w}")

        if x1 > 320 and x2 < 960: #middle 
            if x2 <= 600: #left middle
                if y1 <= 300: #middle top
                    #print("middle: top left")
                    w = w *1.2
                elif y1 <= 480: #middle middle
                    #print("middle: middle left")
                    w = w *1.1
            elif x1 >= 720: #middle right
                if y1 <= 300: #middle top
                    #print("middle: top right")
                    w = w *1.35
                elif y1 <= 480: #middle middle
                    #print("middle: middle right")
                    w = w *1.2
            else: #middle too far
                if y1 <= 300: #right middle far
                    if pixel_h <= 25 and pixel_h >=20:
                        #print("middle: far small")
                        w = w *1.15
                    elif pixel_h <=50 and pixel_h >=35:
                        #print("middle: far medium")
                        w = w *1.25
                elif y1 >= 300 and y1 <= 480:
                    
                    if pixel_w >= 48 and pixel_h >=38:
                        #print("middle: middle big")
                        w = w*1.17
                    elif pixel_w <= 34 and pixel_h <= 27:
                        #print("middle: middle small")
                        w = w*1.17
                    else:
                        #print("None")
                        w = w*1.17
        pred_z0 = w * fx/pixel_w
        pred_x0 = ((x1 + pixel_w - 660) * pred_z0/fx) - w
        diff_x0 = x0 - pred_x0
        diff_z0 = z0-pred_z0

        if abs(diff_z0)/z0 < 0.15:
            error_z0 +=1
        if abs(diff_x0)/x0 < 0.15:
            error_x0 +=1
        
        
            
        diff_x0_list.append(abs(diff_x0))
        diff_list.append(abs(diff_z0))
                        
    
    return (diff_list,diff_x0_list,error_z0, error_x0)
       


In [465]:

avg_error_training_z0_10 = []
avg_error_training_x0_10 = []
for j in range(10):
    diff = []
    diff_x0 = []
    count_z0 = 0
    count_x0 = 0
    for i in range(len(images_training_paths)):
        gt_temp = gt[i]
        cars_info = predict_distance_probability(gt_temp,1)

        diff.extend(cars_info[0])
        diff_x0.extend(cars_info[1])
        count_z0 += cars_info[2]
        count_x0 += cars_info[3]
    avg_error_training_z0_10.append(np.average(diff))
    avg_error_training_x0_10.append(np.average(diff_x0))
    print("Test {0}/10: AVG Error Z0 {1:.2f}, Distance_within_15%_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_15%_rate:{4:.2f}%".format(j+1,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



Test 1/10: AVG Error Z0 4.76, Distance_within_15%_rate: 63.04%, AVG Error X0 0.81, X0_within_15%_rate:75.10%
Test 2/10: AVG Error Z0 4.91, Distance_within_15%_rate: 58.39%, AVG Error X0 0.84, X0_within_15%_rate:73.16%
Test 3/10: AVG Error Z0 4.83, Distance_within_15%_rate: 60.26%, AVG Error X0 0.84, X0_within_15%_rate:74.48%
Test 4/10: AVG Error Z0 4.82, Distance_within_15%_rate: 60.96%, AVG Error X0 0.83, X0_within_15%_rate:73.58%
Test 5/10: AVG Error Z0 4.89, Distance_within_15%_rate: 59.57%, AVG Error X0 0.83, X0_within_15%_rate:73.30%
Test 6/10: AVG Error Z0 4.95, Distance_within_15%_rate: 58.88%, AVG Error X0 0.84, X0_within_15%_rate:73.72%
Test 7/10: AVG Error Z0 4.87, Distance_within_15%_rate: 61.86%, AVG Error X0 0.83, X0_within_15%_rate:74.13%
Test 8/10: AVG Error Z0 4.77, Distance_within_15%_rate: 59.99%, AVG Error X0 0.85, X0_within_15%_rate:73.86%
Test 9/10: AVG Error Z0 4.84, Distance_within_15%_rate: 61.03%, AVG Error X0 0.82, X0_within_15%_rate:74.83%
Test 10/10: AVG Err

## Determine the optimal value of p that minimise the 2nd value based on the current optimal Model

In [475]:

for p in [600,610,620,630,640,650,660,670,680,690,700,710,720]:
    
    diff = []
    diff_x0 = []
    count_z0 = 0
    count_x0 = 0
    #print(f"p = {p}")
    for i in range(len(images_training_paths)):
        gt_temp = gt[i]

        cars_info = predict_distance_6(gt_temp,p,flag = 1)

        diff.extend(cars_info[0])
        diff_x0.extend(cars_info[1])
        count_z0 += cars_info[2]
        count_x0 += cars_info[3]
    
    print("Test {0}/10: AVG Error Z0 {1:.2f}, Distance_within_15%_rate: {2:.2f}%, AVG Error X0 {3:.2f}, X0_within_15%_rate:{4:.2f}%".format(p,np.average(diff),100 * count_z0/len(diff),np.average(diff_x0),100 * count_x0/len(diff_x0)))



Test 600/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 3.51, X0_within_15%_rate:66.85%
Test 610/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 3.03, X0_within_15%_rate:67.13%
Test 620/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 2.54, X0_within_15%_rate:67.75%
Test 630/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 2.06, X0_within_15%_rate:68.59%
Test 640/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 1.59, X0_within_15%_rate:69.63%
Test 650/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 1.13, X0_within_15%_rate:71.15%
Test 660/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 0.70, X0_within_15%_rate:71.91%
Test 670/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 0.48, X0_within_15%_rate:74.06%
Test 680/10: AVG Error Z0 2.46, Distance_within_15%_rate: 75.94%, AVG Error X0 0.57, X0_within_15%_rate:82.04%
T